In [ ]:
import polars as pl
import pickle

In [ ]:
data = pl.read_csv('./data/ramirez2022/RAMIREZ2022.csv', has_header=True, separator=';')

In [ ]:
from urllib.request import urlopen
from urllib.parse import quote
from time import sleep


def to_InChI(ids: str) -> str:
    try:
        url = (
            "https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/name/"
            + quote(ids)
            + "/property/InChI/TXT"
        )
        ans = urlopen(url).read().decode("utf8").rstrip()
        ans = ans.split("\n")[0]

    except:
        try:
            url = (
                "https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/cid/"
                + quote(ids)
                + "/property/InChI/TXT"
            )
            ans = urlopen(url).read().decode("utf8").rstrip()
            ans = ans.split("\n")[0]
        except:
            print("not ok:", url)
            ans = None

    sleep(0.1)
    return ans

In [ ]:
inchis = data['name'].apply(to_InChI).rename('inchi')

In [ ]:
inchis.describe()

In [ ]:
data = data.with_columns(inchis)

In [ ]:
data.write_parquet("./data/ramirez2022/raw/data.parquet")

In [ ]:
pcpck = pl.read_json('./data/pcsaft_pck/all_pcsaft_fluids.json')
kij = pl.read_json('./data/pcsaft_pck/mixture_binary_pairs_pcsaft.json')
habicht = pl.read_csv('./data/Habicht/data.csv', sep=';')

In [ ]:
hsmiles = habicht['name'].apply(to_InChI)

In [ ]:
habicht=habicht.with_columns(hsmiles.rename('inchi'))

In [ ]:
pcpcksmiles = pcpck['CAS'].apply(to_InChI)

In [ ]:
pcpck = pcpck.with_columns(pcpcksmiles.rename('inchi'))

In [ ]:
df1 = habicht[['name','inchi', 'm','sigma', 'u']]

In [ ]:
df2 = pcpck[['name','inchi', 'm','sigma', 'u','volA','uAB', 'dipm', 'dipnum']]

In [ ]:
df = pl.concat([df1,df2], how='diagonal').fill_null(0)

In [ ]:
kij = kij.filter(kij['kijT'].is_null())

In [ ]:
kijsmiles = kij[["CAS1", "CAS2"]].apply(lambda x: (to_InChI(x[0]), to_InChI(x[1])))

In [ ]:
kij.write_parquet('../ePC-SAFT/data/parameters/raw/kij.parquet')
df.write_parquet('../ePC-SAFT/data/parameters/raw/pure.parquet')